## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True

tradelogger.setLevel(logging.DEBUG)

In [2]:
tradelogger.info("Let's do trading")

In [7]:
pd.set_option('display.max_rows', None)

##  <span style="color:green">Trading Strategy </span>

In [4]:
from talib import MIN, MAX, CDLCLOSINGMARUBOZU,CDL3WHITESOLDIERS,CDLMORNINGSTAR,CDLMORNINGDOJISTAR,CDL3LINESTRIKE,CDL3OUTSIDE,CDLENGULFING,CDLBELTHOLD,CDLABANDONEDBABY,CDL3INSIDE,CDLPIERCING,CDLBREAKAWAY,CDLXSIDEGAP3METHODS,CDLHAMMER,CDLMARUBOZU,CDL3BLACKCROWS,CDLIDENTICAL3CROWS,CDLEVENINGSTAR,CDLEVENINGDOJISTAR,CDLDARKCLOUDCOVER,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLSPINNINGTOP,CDLGRAVESTONEDOJI,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI
cdlfunc_array = [CDLCLOSINGMARUBOZU,CDL3WHITESOLDIERS,CDLMORNINGSTAR,CDLMORNINGDOJISTAR,CDL3LINESTRIKE,CDL3OUTSIDE,CDLENGULFING,CDLBELTHOLD,CDLABANDONEDBABY,CDL3INSIDE,CDLPIERCING,CDLBREAKAWAY,CDLXSIDEGAP3METHODS,CDLHAMMER,CDLMARUBOZU,CDL3BLACKCROWS,CDLIDENTICAL3CROWS,CDLEVENINGSTAR,CDLEVENINGDOJISTAR,CDLDARKCLOUDCOVER,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLSPINNINGTOP,CDLGRAVESTONEDOJI,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI]

def detectCDPattern(prices, a, noofcandles=15, strPlot="engulfing"):
    #https://github.com/mrjbq7/ta-lib
    
    annotateText = []
    annotateIndex = []
    
    for index, curr in prices.iloc[noofcandles:].iterrows():
        text = ""
        
        index_r = prices.index.get_loc(index)+1
        index_l = max(0, index_r - noofcandles)
        #rice = pd.DataFrame()
        popen = prices.iloc[index_l:index_r]['open']
        phigh = prices.iloc[index_l:index_r]['high']
        pclose = prices.iloc[index_l:index_r]['close']
        plow = prices.iloc[index_l:index_r]['low']
        
        function = CDLDOJI
        
        for cdlfunc in cdlfunc_array:
            ret_val = cdlfunc(popen, phigh, plow, pclose)[-1]
            if (ret_val > 0):
                text += "Bull: "+re.sub('CDL', '', cdlfunc.__name__)+"<br>"
            elif (ret_val < 0):
                text += "Sell: "+re.sub('CDL', '', cdlfunc.__name__)+"<br>"
                
        if text != "":
            annotateText.append(text)
            annotateIndex.append(index)
            
            a.appendHoverText(index,text)
            #logger.debug(index.strftime("%d-%m-%y %H:%M")+": "+text)
    


In [8]:
# ====== Tradescript Wrapper =======
# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a,b: ROC(a, b) >= 0.1
#TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1

import traceback
TREND_UP = lambda a,b: a > MAX(REF(a,1),b)
TREND_DOWN = lambda a,b: a < MIN(REF(a,1),b)

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
blackoutEnabled = True

blackoutFrom = '11:30:00'
blackoutTo = '12:30:00'

blackout = lambda a,x,y=blackoutFrom,z=blackoutTo: a[blackoutEnabled & (a.index >= x) & (a.index < x+' '+y) | (a.index > x+' '+z)]


        
class algoTrade:
   
    def __init__(s, symbol):
        logger.debug("AlgoTrade Called: "+symbol)
        
        #s.symbol = symbol
        s.STATE = "WAIT"
        s.stoploss = 0
        s.target = 0
        s.price = 0
        s.trigger = 0
        s.quantity = 0
        s.tag = "bot"
        s.profit = 0
  

    def checkZone(s, a):
        s.zone1 = a.replace( hour=9, minute=15, second=0,microsecond=0)
        s.zone2 = a.replace(hour=11, minute=30, second=0,microsecond=0)
        s.zone3 = a.replace(hour=12, minute=30, second=0,microsecond=0)
        s.zone4 = a.replace(hour=14, minute=45, second=0,microsecond=0)

        text = ""
        if a < s.zone1:
            text = "zone1"
        elif a <= s.zone2:
            text = "zone2"
        elif a <= s.zone3:
            text = "zone3"
        elif a <= s.zone4:
            text = "zone4"
        else:
            text = "zone5"

        return text

    def getHoverText(s,index):
        iloc = s.CLOSE.index.get_loc(index)
        return s.hovertextarray.iloc[iloc][0]
        
    def appendHoverText(s, index,text):
        #logger.debug(str(index)+": "+text)
        iloc = s.CLOSE.index.get_loc(index)
        s.hovertextarray.iloc[iloc] = s.hovertextarray.iloc[iloc] + text
    
    # Long Strategies
    def long_bb(s):
        temp_df = 0
        #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        #temp_df = temp_df | (REF(s.rsi,1) <=25)
        #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        temp_df = temp_df & (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)  
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_bb(s):
        temp_df = 0
        #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        #temp_df = temp_df | (REF(s.rsi,1) >=75)
        #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        #temp_df = temp_df | CROSSOVER(s.BBT, s.OPEN)
        #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df & (s.CLOSE.shift(1) == s.max.shift(1)) & (s.CLOSE < s.OPEN)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    
    def long_pa(s):
        temp_df = False
        temp_df = temp_df | (CDLMORNINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLMORNINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3WHITESOLDIERS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        
        temp_df = temp_df | (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3LINESTRIKE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3OUTSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLABANDONEDBABY(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | ((CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) & TREND_DOWN(s.CLOSE,4))
        
        temp_df = temp_df | CDL3INSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLPIERCING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHAMMER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) 
        temp_df = temp_df | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) ==100)

        
        #temp_df = TREND_DOWN(s.OPEN,5) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(s):
        temp_df = False
        
        temp_df = temp_df | (CDL3BLACKCROWS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLIDENTICAL3CROWS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDL3LINESTRIKE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        temp_df = temp_df | (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDL3OUTSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLABANDONEDBABY(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDARKCLOUDCOVER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | (CDLHANGINGMAN(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) 
        temp_df = temp_df | (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        #temp_df = TREND_UP(s.CLOSE,5) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_breakout(s):
        temp_df = s.haCLOSE >= s.BBT.shift(1)
        temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haOPEN <= s.BBB.shift(1)
        temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_ha(s):
        temp_df = (s.haCLOSE >= REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) #& (REF(s.CLOSE,1) < REF(s.OPEN,1))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (s.haCLOSE <= REF(s.haOPEN,1)) & (s.haOPEN > s.haCLOSE) #& (REF(s.CLOSE,1) > REF(s.OPEN,1))
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s, price_ohlc_df):
        global blackoutEnabled
        
        #logger.info(price_ohlc_df.index[-1])
        s.price_df = price_ohlc_df
        s.OPEN = price_ohlc_df['open']
        s.CLOSE = price_ohlc_df['close']
        s.HIGH = price_ohlc_df['high']
        s.LOW = price_ohlc_df['low']
    
        s.hovertextarray = pd.DataFrame(np.full(price_ohlc_df.index.shape[0],""))
        
        #Heikin Asi
        s.haOPEN  = (s.OPEN.shift(1) + s.CLOSE.shift(1))/2
        s.haHIGH  = pd.DataFrame([s.HIGH,s.OPEN,s.CLOSE]).max(axis = 0, skipna = True)
        s.haLOW   = pd.DataFrame([s.HIGH,s.LOW,s.CLOSE]).min(axis = 0, skipna = True)
        s.haCLOSE = (s.OPEN+s.HIGH+s.LOW+s.CLOSE)/4
        
        s.BBT, s.BBM, s.BBB = BBANDS( s.CLOSE, 20,2,2,0)
        #s.BBT2, s.BBM2, s.BBB2 = BBANDS( s.CLOSE, 20,3,3,0)
        s.macd, s.macdsignal, s.macdhist = MACDEXT(s.CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        s.SD = STDDEV(s.CLOSE)
        s.fastk, s.fastd = STOCHF(s.HIGH, s.LOW, s.CLOSE)
        s.rsi = RSI(s.CLOSE, timeperiod=20)
        s.min = MIN(s.CLOSE, timeperiod=30)
        s.max = MAX(s.CLOSE, timeperiod=30)
        
        
        s.buy  = s.long_pa()
        s.sell = s.short_pa()
                
        s.buy['low'] = s.LOW
        s.sell['low'] = s.LOW
        s.buy['high'] = s.HIGH
        s.sell['high'] = s.HIGH
        s.buy['close'] = -1 * s.CLOSE
        s.sell['close'] = s.CLOSE
        
        #if s.buy['buy'].any():
        #    s.buy = blackout(s.buy,s.buy.index[-1].date().strftime("%Y-%m-%d"))
        #if s.sell['sell'].any():
        #    s.sell = blackout(s.sell, s.sell.index[-1].date().strftime("%Y-%m-%d"))
        
        s.buy = s.buy[s.buy['buy']]
        s.sell = s.sell[s.sell['sell']]
        
        return (s.buy,s.sell)
    
    def trade_manager(s, symbol, price_ohlc_df): 
        #logger.info(price_ohlc_df.tail())
            
        s.symbol = symbol
        s.tradeDecision(price_ohlc_df)
        
        logger.debug(s.buy)
        logger.debug(s.sell)
        
        s.timestamp = price_ohlc_df.index[-1].replace(second=0, microsecond=0)
        
        try:
            buy  = (s.timestamp - s.buy.index[-1]).seconds
        except:
            buy = 360
            
        try:
            sell = (s.timestamp - s.sell.index[-1]).seconds
        except:
            sell = 360
        
        logger.info(str(s.timestamp)+":"+str(buy)+", "+str(sell))
        s.trade_setup(s.CLOSE.iloc[-1])
        
        # Lunch Time
        s.zone = s.checkZone(s.timestamp)
        
        if(s.zone=="zone3" and s.checkZone(s.CLOSE.index[-2])=="zone2"):
            return(s.place_order("SQUAREOFF"))
        
        # Too close to closing bell
        #if(zone=="zone5" and s.checkZone(s.CLOSE.index[-2])=="zone4"):
        #    return(s.place_order("SQUAREOFF"))
        
        if(s.zone == "zone3"):
            return 0            
        
        if buy < 60:
            return(s.place_order("BUY"))
        elif sell <60:
            return(s.place_order("SELL"))
        
        
        return 0

    
    
    def trade_setup(s, price):
        s.price = price
        s.trigger = price
        s.stoploss = toTick(price * 0.1/100)
        s.target = toTick(price * 1/100)
        s.quantity = 1
        s.tag = "bot"
        
        #logger.debug(s.price)
    
    def place_order(s, action):
        
        #trade_setup()
        #zone = s.checkZone(s.CLOSE.index[-1])
        
        fbuy = lambda:   buy_bo(symbol=s.symbol, price=s.price, trigger=s.trigger, stoploss=s.stoploss, squareoff=s.target, quantity=s.quantity, tag=s.tag)
        fsell = lambda: sell_bo(symbol=s.symbol, price=s.price, trigger=s.trigger, stoploss=s.stoploss, squareoff=s.target, quantity=s.quantity, tag=s.tag)
        fsquareoff = lambda: squareoff(symbol=s.symbol, tag="bot")
        margin = s.quantity * s.price
        
        if s.zone == "zone5":
            action = "SQUAREOFF"
            
        logger.info("Place Order["+s.timestamp.strftime("%H:%M")+"]: "+s.STATE+"=>"+action)
                
        if s.STATE == "WAIT":
            if action == "BUY":
                fbuy()
                s.profit = s.profit - margin
                s.STATE = "BUY"
            elif action == "SELL":
                fsell()
                s.profit = s.profit + margin
                s.STATE = "SELL"
        elif s.STATE == "BUY":
            if action == "SELL":
                fsquareoff()
                fsell()
                s.profit = s.profit + 2 * margin
                s.STATE = "SELL"
            elif action == "SQUAREOFF":
                fsquareoff()
                s.profit = s.profit + margin
                s.STATE = "WAIT"
        elif s.STATE == "SELL":
            if action == "BUY":
                fsquareoff()
                fbuy()
                s.profit = s.profit - 2 * margin
                s.STATE = "BUY"
            elif action == "SQUAREOFF":
                fsquareoff()
                s.profit = s.profit - margin
                s.STATE = "WAIT"
                
        return s.profit

        
    def plot(s, symbol, noofcandles=120, ha=False):
        init_notebook_mode(connected=True)
        fig = tools.make_subplots(rows=4, cols=1, shared_xaxes=True, row_width=[1,1,1,3], vertical_spacing = 0.05)
        fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
        fig['layout'].update(height=750, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
        
        fig['layout']['yaxis']['anchor'] = 'x'
        fig['layout']['yaxis']['side'] = 'right'

        fig['layout']['xaxis']['rangeselector'] = dict(
                    buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                                  dict(count=2, label='2h', step='hour', stepmode='backward'),
                                  dict(count=3, label='3h', step='hour', stepmode='backward'),
                                  dict(count=6, label='4h', step='hour', stepmode='backward'),
                                  dict(count=6, label='1d', step='hour', stepmode='backward'),
                                  dict(step='all')]))
        
        yMin = s.LOW.iloc[-1*noofcandles:-1].min()*0.99
        yMax = s.HIGH.iloc[-1*noofcandles:-1].max()*1.001

        xMin = s.CLOSE.index[-1*noofcandles]
        xMax = s.CLOSE.index[-1]
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['xaxis']['range'] = [xMin, xMax]
        
        #print(s.buy.shape)
        #print(s.sell.shape)
        
        
        #print("plot")
        
        traceBuy=go.Scatter(x=s.buy.index.astype('str'), y=-1 * s.buy.close, name='BUY', mode="markers",marker = dict(color = 'rgba(119, 221, 119, 0.2)', size = 25, symbol='circle', line = dict(color = 'rgb(119, 221, 119)', width = 3)),showlegend=False)
        
        traceSell=go.Scatter(x=s.sell.index.astype('str'), y=s.sell.close, name='SELL', mode="markers",marker = dict(color = 'rgba(255, 0, 0,0.2)',size = 25, symbol='circle', line = dict(color = 'rgb(255, 0, 0)', width = 3)),showlegend=False)
       
        if ha:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.haOPEN, high=s.haHIGH, low=s.haLOW, close=s.haCLOSE, name="Heikin Ashi", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)        
        else:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.OPEN, high=s.HIGH, low=s.LOW, close=s.CLOSE, name="Candlestick", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)      
            
        traceMACD = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macd, name='MACD', line=dict(color='black'),showlegend=False)
        traceMACDSignal = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
        traceMACDHist = go.Bar(x=s.CLOSE.index.astype('str'), y=s.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        traceSK  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastk, name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
        traceSD  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastd, name='%D', line=dict(color='red'),showlegend=False)
        traceBBT = go.Scatter(x=s.BBT.index.astype('str'), y=s.BBT, name='BB_up',  line=dict(color='lightgrey'),showlegend=False, hoverinfo = 'none')
        traceBBB = go.Scatter(x=s.BBB.index.astype('str'), y=s.BBB, name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        traceBBM = go.Scatter(x=s.BBM.index.astype('str'), y=s.BBM, name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        
        traceMIN = go.Scatter(x=s.min.index.astype('str'), y=s.min, name='MIN',  line=dict(color='darkblue'), showlegend=False, hoverinfo = 'none')
        traceMAX = go.Scatter(x=s.max.index.astype('str'), y=s.max, name='MAX',  line=dict(color='darkcyan'), showlegend=False, hoverinfo = 'none')
        
        traceRSI = go.Scatter(x=s.rsi.index.astype('str'), y=s.rsi, name='RSI',  line=dict(color='darkcyan'), showlegend=False)
        
        fig.append_trace(traceBBT, 1, 1)
        fig.append_trace(traceBBB, 1, 1)
        fig.append_trace(traceBBM, 1, 1)
        fig.append_trace(traceBuy, 1, 1)
        fig.append_trace(traceSell, 1, 1)
        fig.append_trace(traceMIN, 1, 1)
        fig.append_trace(traceMAX, 1, 1)
    
        fig.append_trace(traceSK, 2, 1)
        fig.append_trace(traceSD, 2, 1)
        
        
        
        fig['layout']['yaxis2']['title']="Stochastics"
        fig['layout']['yaxis2']['side']="right"
        fig['layout']['yaxis2']['anchor']="x"
        fig['layout']['yaxis2']['tickvals']=[20,80,30,70]
        
        fig.append_trace(traceMACD, 3, 1)
        fig.append_trace(traceMACDSignal, 3, 1)
        fig.append_trace(traceMACDHist, 3, 1)
        fig.append_trace(trace, 1, 1)
        
        fig['layout']['yaxis3']['anchor']="x"
        fig['layout']['yaxis3']['side']="right"
        fig['layout']['yaxis3']['title']="MACD"
        fig['layout']['yaxis1']['title']="Candlestick"
        
        
        fig.append_trace(traceRSI, 4, 1)
        fig['layout']['yaxis4']['title']="RSI"
        fig['layout']['yaxis4']['side']="right"
        fig['layout']['yaxis4']['anchor']="x"
        fig['layout']['yaxis4']['tickvals']=[30,50,70]
        
        iplot(fig, filename="plot/"+x+".html")
        


def trade_simulator(portfolio, toDate):
    open("log/live_log.log", "w")
    
    fromDate =   getFromDate(toDate, days=4)
    logger.setLevel(logging.CRITICAL)
    papertrade = True
    
    cumsum = 0
    tradelogger.setLevel(logging.INFO)
    for i,y in portfolio.iterrows():
        logger.info(y[0])
    
        temp_data = getData(y[0], fromDate, toDate, "minute",force =True)
        
        #print(temp_data.head())

        a = algoTrade(y[0])
        a.tradeDecision(temp_data)
        
        buysell = pd.concat([a.buy.close, a.sell.close]).sort_index()
            
        temp_index = toDate.replace(hour=11,minute=31)
        temp_df = pd.DataFrame(data=[temp_data.loc[temp_index,'close']],index=[temp_index])
        buysell = buysell.append(temp_df)
        
        temp_index = toDate.replace(hour=15,minute=1)
        temp_df = pd.DataFrame(data=[temp_data.loc[temp_index,'close']],index=[temp_index])
        buysell= buysell.append(temp_df)
        
        
        buysell = buysell.sort_index()
        
        if not buysell.empty:
            buysell = buysell[buysell.index>buysell.index[-1].date().strftime("%Y-%m-%d")]
        
        #print(buysell)
        
        payoff = 0
        if portfolio.shape[0] == 1:
            #detectCDPattern(temp_data, a)
            pass
        
        previndex = buysell.index[0].replace(minute=14)
        #print(previndex)
        for index in buysell.index:
            if index == previndex:
                continue
            r = temp_data.index.get_loc(index)+1
            l = r - 30
            
            l2 = r - int(np.floor((index - previndex).total_seconds()/60))
            
            if l2 != r and True:
                low = min(temp_data.iloc[l2:r]['low'])
                high = max(temp_data.iloc[l2:r]['high'])
                
                if (a.price - a.stoploss) > low and a.STATE == "BUY":
                    a.profit = a.profit + (a.price - a.stoploss)
                    logger.info("SO S:"+a.STATE+":"+str(a.profit)+","+str(a.price - a.stoploss))
                    a.STATE = "WAIT"
                elif  (a.price + a.stoploss) < high and a.STATE == "SELL":
                    a.profit = a.profit - (a.price + a.stoploss)
                    logger.info("SO B:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.stoploss))
                    a.STATE = "WAIT"
                elif  (a.price + a.target) < high and a.STATE == "BUY":
                    a.profit = a.profit + (a.price + a.target)
                    logger.info("PB S:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.target))
                    a.STATE = "WAIT"
                elif  (a.price - a.target) > low and a.STATE == "SELL":
                    a.profit = a.profit - (a.price - a.target)
                    logger.info("PB B:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.target))
                    a.STATE = "WAIT"
            
            #print(low, high)
            previndex = index
            
            a.zone = a.checkZone(index)
            
            a.trade_manager(y[0], temp_data.iloc[l:r])
            logger.info(a.profit)
            
            #print(profit)
            
        if a.STATE == "BUY":
            a.profit = a.profit + temp_data.close[-1]
            
        elif a.STATE == "SELL":
            a.profit = a.profit - temp_data.close[-1]
        
        print(y[0],"\t",a.profit, "\t",str(round(a.profit/a.CLOSE[-1]*100,2))+"%" )
        logger.info(y[0]+": "+str(a.profit)+" ("+str(round(a.profit/a.CLOSE[-1]*100,2))+"%)")
    
        cumsum = cumsum + round(a.profit/a.CLOSE[-1]*100,2)
        if portfolio.shape[0] == 1:
            #a.plot(x, 360, False)
            pass
            
        #break
    

if mode == "Algo":
    gc.collect()
    x = 'INFRATEL'
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    trade_simulator(portfolio, toDate)

                          0
2019-04-30 09:15:00  262.70
2019-04-30 09:16:00 -264.05
2019-04-30 09:17:00  263.75
2019-04-30 09:19:00 -263.90
2019-04-30 09:19:00  263.90
2019-04-30 09:21:00  263.25
2019-04-30 09:31:00  263.05
2019-04-30 09:32:00 -263.45
2019-04-30 09:34:00  262.45
2019-04-30 09:37:00 -262.95
2019-04-30 09:41:00 -263.05
2019-04-30 09:45:00 -263.00
2019-04-30 09:45:00  263.00
2019-04-30 09:49:00  262.15
2019-04-30 09:50:00 -262.00
2019-04-30 09:51:00  261.60
2019-04-30 09:52:00  261.45
2019-04-30 09:52:00 -261.45
2019-04-30 09:58:00 -261.30
2019-04-30 09:59:00 -261.80
2019-04-30 10:00:00  261.55
2019-04-30 10:01:00  261.20
2019-04-30 10:09:00 -262.45
2019-04-30 10:13:00  262.45
2019-04-30 10:21:00  263.05
2019-04-30 10:22:00  262.55
2019-04-30 10:24:00  262.35
2019-04-30 10:25:00 -262.45
2019-04-30 10:26:00 -262.90
2019-04-30 10:29:00 -262.50
2019-04-30 10:30:00 -262.85
2019-04-30 10:34:00  263.15
2019-04-30 10:37:00  264.00
2019-04-30 10:38:00  263.75
2019-04-30 10:39:00 

- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)


In [17]:
def backtest(portfolio, toDate, plot=False):
    open("log/live_log.log", "w")
    
    fromDate =   getFromDate(toDate, days=4)
    logger.setLevel(logging.CRITICAL)
    papertrade = True
    
    cumsum = 0
    tradelogger.setLevel(logging.INFO)
    for i,y in portfolio.iterrows():
        logger.info(y[0])
    
        temp_data = getData(y[0], fromDate, toDate, "minute",force =True)
        
        #print(temp_data.head())

        a = algoTrade(y[0])
        a.symbol = y[0]
        a.tradeDecision(temp_data)
        
        buysell = pd.concat([a.buy.close, a.sell.close]).sort_index()
            
        temp_index = toDate.replace(hour=11,minute=31)
        temp_df = pd.DataFrame(data=[temp_data.loc[temp_index,'close']],index=[temp_index])
        buysell = buysell.append(temp_df)
        
        temp_index = toDate.replace(hour=15,minute=1)
        temp_df = pd.DataFrame(data=[temp_data.loc[temp_index,'close']],index=[temp_index])
        buysell= buysell.append(temp_df)
        
        
        buysell = buysell.sort_index()
        
        if not buysell.empty:
            buysell = buysell[buysell.index>buysell.index[-1].date().strftime("%Y-%m-%d")]
        
        
        #print(buysell)
        
        payoff = 0
        if portfolio.shape[0] == 1 and plot:
            detectCDPattern(temp_data, a=a)
            pass
        
        previndex = buysell.index[0].replace(minute=14)
        #print(previndex)
        for index in buysell.index:
            if index == previndex:
                #print(buysell.loc[index])
                continue
                
            a.timestamp = index
            r = temp_data.index.get_loc(index)+1
            l = r - 30
            
            l2 = r - int(np.floor((index - previndex).total_seconds()/60))
            
            if l2 != r and True:
                low = min(temp_data.iloc[l2:r]['low'])
                high = max(temp_data.iloc[l2:r]['high'])
                
                if (a.price - a.stoploss) > low and a.STATE == "BUY":
                    a.profit = a.profit + (a.price - a.stoploss)
                    logger.info("SO S:"+a.STATE+":"+str(a.profit)+","+str(a.price - a.stoploss))
                    a.STATE = "WAIT"
                elif  (a.price + a.stoploss) < high and a.STATE == "SELL":
                    a.profit = a.profit - (a.price + a.stoploss)
                    logger.info("SO B:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.stoploss))
                    a.STATE = "WAIT"
                elif  (a.price + a.target) < high and a.STATE == "BUY":
                    a.profit = a.profit + (a.price + a.target)
                    logger.info("PB S:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.target))
                    a.STATE = "WAIT"
                elif  (a.price - a.target) > low and a.STATE == "SELL":
                    a.profit = a.profit - (a.price - a.target)
                    logger.info("PB B:"+a.STATE+":"+str(a.profit)+","+str(a.price + a.target))
                    a.STATE = "WAIT"
                        
            # Lunch Time
            a.zone = a.checkZone(index)
            
            if(a.zone=="zone3" and a.checkZone(previndex)=="zone2"):
                temp_price = temp_data.loc[index.replace(minute=31),'close']
                a.trade_setup(temp_price)
                a.place_order("SQUAREOFF")
                previndex = index
                continue

            # Too close to closing bell
            #if(zone=="zone5" and s.checkZone(s.CLOSE.index[-2])=="zone4"):
            #    return(s.place_order("SQUAREOFF"))

            if(a.zone == "zone3"):
                previndex = index
                continue  
                
            previndex = index
            
            temp_price = buysell.loc[index].values
            
            if type(temp_price[0]).__name__ == 'ndarray':
                temp_price = -1 * abs(temp_price[0]) #if both buy and sell then do buy
                #print(temp_price)
                
            temp_price = temp_price[0]
            
            if temp_price < 0:
                a.trade_setup(-1*temp_price)
            else:
                a.trade_setup(temp_price)

            if temp_price < 0:
                a.place_order("BUY")
            elif temp_price > 0:
                a.place_order("SELL")
                
            logger.info(a.profit)
            
        if a.STATE == "BUY":
            a.profit = a.profit + temp_data.close[-1]
            
        elif a.STATE == "SELL":
            a.profit = a.profit - temp_data.close[-1]
        
        print(y[0],"\t",a.profit, "\t",str(round(a.profit/a.CLOSE[-1]*100,2))+"%" )
        logger.info(y[0]+": "+str(a.profit)+" ("+str(round(a.profit/a.CLOSE[-1]*100,2))+"%)")
    
        cumsum = cumsum + round(a.profit/a.CLOSE[-1]*100,2)
        if portfolio.shape[0] == 1 and plot:
            a.plot(x, 360, False)
            pass
            
        #break
    

if mode == "Algo":
    x = 'TCS'
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    backtest(portfolio, toDate)

TCS 	 4.749999999996362 	 0.21%
